In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 100)

In [3]:
import seaborn as sns

/home/maciek/miniconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
from matplotlib.pylab import plt

In [5]:
%matplotlib inline

In [6]:
# raport = pd.concat(
#     (pd.read_csv('dude_horizontal_dock_final.csv', index_col=0),
#     pd.read_csv('dude_horizontal_dude_final.csv', index_col=0),
#     pd.read_csv('dude_horizontal_vina_final.csv', index_col=0))).reset_index(drop=True)

In [10]:
raport_per_target = pd.read_csv('dude_per_target.csv', index_col=0)
raport_per_target['n_folds'] = 5
raport_per_target['kind'] = 'Per Target'

raport_per_target['engine'].replace('dock', 'Dock 6.6', inplace=True)
raport_per_target['engine'].replace('dude', 'Dock 3.6', inplace=True)
raport_per_target['engine'].replace('vina', 'Autodock Vina', inplace=True)

raport_per_target['v'] = 'RF-Score-VS v' + raport_per_target['v'].astype(str)
raport_per_target.rename(columns={'v':'score'}, inplace=True)

raport_per_target = raport_per_target.groupby(('engine', 'kind', 'score', 'n_folds', 'dude_id')).mean().reset_index()

In [11]:
raport_horizontal_multitarget = pd.read_csv('dude_horizontal_multitarget.csv', index_col=0)
# raport_horizontal = pd.read_csv('dude_horizontal_final.csv', index_col=0)

raport_horizontal_multitarget.drop('head',axis=1, inplace=True)

# rename 'v' to 'scores'
raport_horizontal_multitarget['v'] = 'RF-Score-VS v' + raport_horizontal_multitarget['v'].astype(int).astype(str)
raport_horizontal_multitarget.rename(columns={'v':'score'}, inplace=True)
raport_horizontal_multitarget['n_folds'] = 5

raport_horizontal_multitarget['kind'] = 'Horizontal Multitarget'

raport_horizontal_multitarget['engine'].replace('dock', 'Dock 6.6', inplace=True)
raport_horizontal_multitarget['engine'].replace('dude', 'Dock 3.6', inplace=True)
raport_horizontal_multitarget['engine'].replace('vina', 'Autodock Vina', inplace=True)

In [15]:
#raport_final.groupby(('engine', 'kind', 'score', 'n_folds')).std()

In [20]:
dude_ids = ['aa2ar', 'abl1', 'ace', 'aces', 'ada', 'ada17', 'adrb1', 'adrb2', 'akt1', 'akt2', 'aldr', 'ampc', 'andr', 'aofb', 'bace1', 'braf', 'cah2', 'casp3', 'cdk2', 'comt', 'cp2c9', 'cp3a4', 'csf1r', 'cxcr4', 'def', 'dhi1', 'dpp4', 'drd3', 'dyr', 'egfr', 'esr1', 'esr2', 'fa10', 'fa7', 'fabp4', 'fak1', 'fgfr1', 'fkb1a', 'fnta', 'fpps', 'gcr', 'glcm', 'gria2', 'grik1', 'hdac2', 'hdac8', 'hivint', 'hivpr', 'hivrt', 'hmdh', 'hs90a', 'hxk4', 'igf1r', 'inha', 'ital', 'jak2', 'kif11', 'kit', 'kith', 'kpcb', 'lck', 'lkha4', 'mapk2', 'mcr', 'met', 'mk01', 'mk10', 'mk14', 'mmp13', 'mp2k1', 'nos1', 'nram', 'pa2ga', 'parp1', 'pde5a', 'pgh1', 'pgh2', 'plk1', 'pnph', 'ppara', 'ppard', 'pparg', 'prgr', 'ptn1', 'pur2', 'pygm', 'pyrd', 'reni', 'rock1', 'rxra', 'sahh', 'src', 'tgfr1', 'thb', 'thrb', 'try1', 'tryb1', 'tysy', 'urok', 'vgfr2', 'wee1', 'xiap']

dude_weights = pd.DataFrame([{'dude_id':dude_id,
                             'size': len(pd.read_csv('head1_full/%s/dude/%s_desc_v%i.csv.gz' % (dude_id, 'actives', 1),
                                                     header=None,
                                                     dtype={i: pd.np.uint16 for i in range(1,37)}).drop_duplicates(0))}
                             for dude_id in dude_ids]
                           )

In [33]:
raport_horizontal_multitarget.groupby(('engine', 'score', 'dude_id')).mean().shape

(849, 28)

In [34]:
sns.set_style("whitegrid")

In [ ]:
data4 = pd.merge(raport_horizontal_multitarget.groupby(('engine', 'score', 'dude_id'), as_index=False).mean(),
                 raport_per_target.groupby(('engine', 'score', 'dude_id'), as_index=False).mean(),
                 on=['dude_id', 'engine', 'score'],
                 suffixes=('_horizontal_multitarget', '_per_target'))
# data4 = pd.merge(data4, pd.read_csv('dude_target_classes.csv'), on='dude_id', how='inner')

# data4['target_class'].fillna('other', inplace=True)

data4 = data4.merge(dude_weights, on='dude_id')
bin_span = 100
data4['size'] = (((data4['size']/bin_span).round())*bin_span).astype(int)
data4['size'] = data4['size'].astype(str) + '-' + (data4['size']+bin_span).astype(str)

# retain Vina only
data4 = data4[data4['engine'] == 'Autodock Vina']

data4.rename(columns={'size': 'Number of active ligands',
                      'engine': 'Docking software',
                      'score': 'Scoring function',
                      'ef1_per_target': 'EF1% [Per-Target]',
                      'ef1_horizontal_multitarget': 'EF1% [Horizontal]',
                     }, inplace=True)

g = sns.FacetGrid(data4,
                  col='Scoring function',
                  row='Docking software',
                  hue='Number of active ligands',
                  hue_order=['0-100',
                             '100-200',
                             '200-300',
                             '300-400',
                             '400-500',
                             '500-600',
                             '600-700',
                            ],
                  sharex=False,
                  sharey=False,
                  margin_titles=True,
                  #hue_kws={'marker': ['^', 'v','s', 'o', 'D']},
                  palette=sns.color_palette("Blues", 7,),
                  #palette=sns.cubehelix_palette(7),
                  size=4,
                  dropna=True,
                 )

g.map(plt.scatter,
      'EF1% [Per-Target]',
      'EF1% [Horizontal]',
      s=40,
      linewidth=3,
      marker='.',
     )


lim = (0, 75)
delta = 5
for ax in g.axes.flat:
    ax.plot(lim, lim, c="r", ls="--", alpha=.5)
    # 5 mark
    ax.plot((lim[0],lim[1]-delta), (lim[0]+delta,lim[1]), c="g", ls=":", alpha=.6)
    ax.plot((lim[0]+delta,lim[1]), (lim[0],lim[1]-delta), c="g", ls=":", alpha=.6)
    # 10 mark
    ax.plot((lim[0],lim[1]-2*delta), (lim[0]+2*delta,lim[1]), c="g", ls=":", alpha=.4)
    ax.plot((lim[0]+2*delta,lim[1]), (lim[0],lim[1]-2*delta), c="g", ls=":", alpha=.4)
g.set(xlim=lim, ylim=lim)
g.add_legend()


# plt.show()

g.savefig('dude_raport_compare_ef1_pertarget_horizontal.png', dpi=300)

In [24]:
for k in ['EF1% [Per-Target]', 'EF1% [Horizontal]']:
    print(data4[k].min(), data4[k].max())

(2.5609686609680002, 80.471874999999997)
(0.0, 74.892432770500008)
